# HMI PINN + Pytorch lightning

mamba create -n nf2test python=3.10
pip install -r requirements.txt
pip

In [ ]:
# general imports
import glob
import os
from datetime import datetime, timedelta
from dateutil.parser import parse
import shutil

# download
import drms
from urllib import request

# data processing
import numpy as np
from sunpy.map import Map
from sunpy.net import Fido
from sunpy.net import attrs as a

# deep learning
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LambdaCallback
import wandb

# NF2
from nf2.train.module import NF2Module, save
from nf2.train.data_loader import SHARPDataModule, SHARPSeriesDataModule
from nf2.data.download import download_HARP_series, find_HARP, download_euv
from nf2.evaluation.unpack import load_cube
from nf2.evaluation.metric import *
from nf2.evaluation.energy import get_free_mag_energy
from nf2.evaluation.series import evaluate_nf2_series
from nf2.evaluation.flares import _calculate_free_energy, get_integrated_euv_map, load_B_map

# visualization
from matplotlib import pyplot as plt

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

In [ ]:
base_path = '02_output/AR7115_20170906_090000/PINN_light'
data_path = '01_input/AR7115_20170906_090000/hmi'

os.makedirs(base_path, exist_ok=True)

save_path = os.path.join(base_path, 'extrapolation_result.nf2')

In [ ]:
bin = 2
spatial_norm = 160
height = 160
b_norm = 2500
d_slice = None 

boundary = {'type': 'potential', 'strides': 4}

data_args = {"data_path": data_path,
             "height":height,
             "spatial_norm": spatial_norm,
             "b_norm": b_norm,
             "work_directory": base_path,
             "bin": bin,
             "Mm_per_pixel": 0.72,
             "slice": d_slice,
             "boundar": boundary
             }

In [ ]:
dim = 256
vector_potential = False

model_args = {"dim": dim, "use_vector_potential": vector_potential}

In [ ]:
lambda_div = 0.1 
lambda_ff = 0.1

iterations = 10000 
iterations = int(iterations)

validation_interval = 1e4 
validation_interval = int(validation_interval)


batch_size = 1e4
batch_size = int(batch_size)

data_args['iterations'] = iterations
data_args['batch_size'] = batch_size
training_args = {"lambda_div": lambda_div,
              "lambda_ff": lambda_ff,}
config = {'data': data_args, 'model': model_args, 'training': training_args}

In [ ]:
sharp_nr = 7115

In [ ]:
wandb_logger = WandbLogger(project='nf2', name=str(sharp_nr), dir=base_path, log_model="all")
wandb_logger.experiment.config.update(config, allow_val_change=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /userhome/jeon_mg/.netrc


In [ ]:
data_module = SHARPDataModule(**data_args)

See https://docs.sunpy.org/en/stable/code_ref/map.html#fixing-map-metadata for how to fix metadata before loading it with sunpy.map.Map.
See https://fits.gsfc.nasa.gov/fits_standard.html forthe FITS unit standards. [sunpy.map.mapbase]


In [ ]:
validation_settings = {'cube_shape': data_module.cube_dataset.coords_shape,
                       'gauss_per_dB': b_norm,
                       'Mm_per_ds': data_module.Mm_per_pixel * spatial_norm}

nf2 = NF2Module(validation_settings, **model_args, **training_args)

In [ ]:
save_callback = LambdaCallback(
    on_validation_end=lambda *args: save(save_path, nf2.model, data_module, config, nf2.height_mapping_model))
checkpoint_callback = ModelCheckpoint(dirpath=base_path,
                                      every_n_train_steps=validation_interval,
                                      save_last=True)

In [ ]:
n_gpus = torch.cuda.device_count()
trainer = Trainer(max_epochs=1,
                  logger=wandb_logger,
                  devices=n_gpus if n_gpus >= 1 else None,
                  accelerator='gpu' if n_gpus >= 1 else None,
                #   strategy='dp' if n_gpus > 1 else None,
                  num_sanity_val_steps=0,
                  val_check_interval=validation_interval,
                  gradient_clip_val=0.1,
                  callbacks=[checkpoint_callback, save_callback], )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(nf2, data_module, ckpt_path='last')

/userhome/jeon_mg/mambaforge/envs/nf2/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:189: UserWarning: .fit(ckpt_path="last") is set, but there is no last checkpoint available. No checkpoint will be loaded.
  rank_zero_warn(


type: Support for `validation_epoch_end` has been removed in v2.0.0. `NF2Module` implements this method. You can use the `on_validation_epoch_end` hook instead. To access outputs, save them in-memory as instance attributes. You can find migration examples in https://github.com/Lightning-AI/lightning/pull/16520.